#  GPT-2 Generation and Fine-Tuning

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-3 model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

# 0. Setup

Run once to install the library

In [1]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

In [2]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [3]:
!nvidia-smi

Mon Feb 14 15:10:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

In [4]:
model_name = "355M" # largest model we can fine-tune

run once to download the file

In [5]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 948Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.20Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 2.03Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:20, 67.9Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.35Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.23Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 5.44Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [6]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-02-14 15:11:53.050151: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-14 15:11:54.112884: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [7]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="Today, I will eat",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Today, I will eat it, but I'll do it cold. I'll eat it cold, but I'll eat it hot. I'll eat it hot, but I'll eat it cold."

This kind of approach will likely be the case as well, because if he wanted to spend more time eating, he would eat more. However, if he wanted to eat more, he would eat more.

On the other hand, if he were to eat less, he would eat less.

However
Today, I will eat a cake for a birthday or anniversary," he said, smiling. "I was surprised how much people liked it."

The cake was inspired by an old friend, Mr. Guccione, who is in his 60s. Mr. Guccione, who is the owner of a bakery in the Bronx, said he went to see the show on Wednesday night and was in the middle of eating when he saw the cake.

"He said, 'Oh my gosh, this
Today, I will eat with my family, go to the movies, and take a shower. I will not be afraid to express myself with my body, my words, and my thoughts. I will not be afraid to say what I believe and what I think. I will not be afraid to make my own 

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [8]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-14 15:13:41.791973: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [9]:
file_name = "yardEps.txt" # your file here

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [10]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=250,
              restore_from='fresh', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=100,
              save_every=200
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-02-14 15:14:44.289999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.85it/s]


dataset has 146492 tokens
Training...
[10 | 25.68] loss=2.61 avg=2.61
[20 | 45.45] loss=2.65 avg=2.63
[30 | 65.23] loss=2.60 avg=2.62
[40 | 84.99] loss=2.47 avg=2.58
[50 | 104.76] loss=2.44 avg=2.55
[60 | 124.55] loss=2.56 avg=2.55
[70 | 144.31] loss=2.48 avg=2.54
[80 | 164.07] loss=2.42 avg=2.53
[90 | 183.83] loss=2.54 avg=2.53
[100 | 203.60] loss=2.48 avg=2.52
======== SAMPLE 1 ========
 talking
26:26
uh we don't need to do
26:28
this because we
26:29
aren't getting paid
26:30
we know what we're getting
26:31
and that we have to go to this
26:33
event and we have to sign up
26:34
and we should give something back
26:36
that's just kind of a [ __ ] thing
26:38
i'd love to see that happening but
26:39
uh and it's a crazy idea
26:41
you know like you've gotta be out of
26:43
your mind at all times
26:45
and you get mad about the whole
26:46
thing but like this thing is a
26:48
cool idea but but like oh well
26:50
it's crazy because you're
26:52
going out to a party
26:53
you'll be like 

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

## Finetune some more

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [27]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-02-10 11:58:52.396304: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14629 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:da:00.0, compute capability: 7.0


To fine-tune some more, run the following. Be sure to increase the number of steps (if it was `500` before, change to `1000` to train for 500 more. the number is cumulative).

In [28]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=500,
              restore_from='latest', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=100,
              save_every=100
              )

Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  6.40it/s]

dataset has 36647 tokens
Training...
Saving checkpoint/run1/model-200


======== SAMPLE 1 ========
 like you don't need the water for the rest of the day
50:39
so you get to say some words like that that
50:45
you are such a sweetie
50:47
you do you guys i'm so excited
50:49
like it's like a dream come true
50:51
and this is like my first time taking this
50:52
guy on a hike i didn't know where he was
50:54
he's just so so nice and i'm just
50:56
like this is my first time you guys have a
51:00
seen me this whole trip i can't believe i
51:02
think you're the most handsome dude you
51:04
see just like i don't know how i feel and
51:06
this is a dream come true thing because
51:08
like i don't know and i feel like yeah i'm
51:10
i'm just like this is not because you're
51:12
that bad a bunch you know how like some
51:14
savage dude you guys know because you're like
51:16
your friend we're trying to get you to
51:18
go join us and we're trying to get you
51:19
to join our team because you're just
51:21
such an awesome guy i can't wait
51:23
i'm just
51:24
thi

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [11]:
gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation

17:20
anonymous
17:21
he is like yeah yeah
17:24
i think he's looking at a map
17:26
of his yard and you know he's thinking
17:28
about like four different people in the yard
17:30
and he he's like that looks a lot like
17:32
technically he's right he's right
17:34
but he's like
17:35
his yard is a lot like like a place
17:37
where you would eat at that time of night
17:39
and then you would like go to the
17:40
house and then you'd go to that yard
17:42
and then you'd eat and then you'd get
17:44
back to it
17:46
and then you'd get home but then
17:48
then you'd like go to bed at night and
17:50
then you'd go to bed at night
17:52
and then you'd go to bed at night
17:54
and then you'd wake up at night and
17:56
then you'd wake up at night and then
17:58
you'd wake up at night and then you'd
18:00
get to bed at night and then you'd get
18:01
to bed at night
18:03
and then you'd get up at night and then
18:05
you'd be like
18:06
sometimes i had to go to bed at night
18:08
because i had 

In [12]:
gpt2.generate(sess, run_name='run1', prefix="00:00")

00:00
so
00:00
it's a
00:01
it's a
00:02
black market for
00:04
sizes that are
00:05
not sold anymore i think
00:07
i've never actually seen a body builder
00:08
make a ton of money i think
00:10
it's just a little bit of a
00:11
disappointment that we're not
00:12
still talking about this
00:14
the body builder market i think
00:15
it's probably lower than it was
00:17
last year because i think some
00:19
of the guys are realizing
00:21
that if they just kind of work
00:23
harder and then they can make
00:25
a little bit more money it will
00:26
also help them be more successful
00:29
and also maybe they'll get a little
00:31
bit more love because i think it
00:33
isn't like guys are like we're
00:35
hustlin and we're doing it to make
00:37
money
00:38
so to not sell out all the time
00:40
it's a little bit more of a
00:42
disappointment because you want
00:44
to sell out but i think the
00:46
market is kind of saturated right
00:48
now i think it's probably more
00:50
so saturated th

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [13]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="00:00",
              nsamples=5,
              batch_size=5
              )

00:00
and i was
05:00
like i'm not june's
05:01
friend
05:02
i'm not gonna wait
05:04
one more week and i'm like
05:06
like it's funny to me that
05:07
you're like i'm a part of your
05:08
life
05:09
they're your friends
05:10
and i'm like i don't want to
05:12
be part of your life anymore
05:13
and i was like whoa
05:15
and i was like all right
05:16
hehe i don't think i need to
05:18
hear you say this anymore but
05:20
yeah i have these
05:22
animals on my computer
05:24
that i play and i have these
05:26
animals in the background that
05:28
wait
05:29
wait all the time
05:31
i have one i will say that it's
05:33
not a good one but i've played
05:34
a lot of
00:00
so i just wanted
00:00
to know that you know we
00:02
had a good time
00:03
in the car
00:05
so i got there i'm like
00:06
i think it was like like a couple
00:07
hours ago you said we were
00:08
together after like two or three
00:09
hours of like getting to know each
00:10
other and yeah
00:11
i did not like it at all i
0

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [15]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      prefix="00:00",
                      nsamples=100,
                      batch_size=20
                      )

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [ ]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name='run1')

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.